In [99]:
import torch
from torch import nn
from torch import optim
from torchvision import transforms
from torchvision import datasets

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
numworkers = 4
batch_size=20

In [100]:
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                         ])
mnist_data = datasets.MNIST('../datasets/', download=True,transform=transform)
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=numworkers)


In [101]:
def LinearBlock(_in,_out,reg=True, inplace=True):
    layers = []
    layers.append(nn.Linear(_in,_out))
    if reg:
        layers.append(nn.ReLU(inplace=inplace))
    return layers

In [102]:
class AdditiveCoupling(nn.Module):
    '''
    Single Layer to be used in NICE. For MNIST we use 5 of these that are rectified.
    '''
    def __init__(self, _in, _out, numLayers=5):
        super(AdditiveCoupling,self).__init__()
        
        self.numLayers = numLayers
        relu = nn.ReLU(inplace=True)

        self.inputLayer = nn.Sequential(nn.Linear(_in,1000),nn.ReLU())
        self.hiddenLayer = nn.Sequential(nn.Linear(1000,1000),nn.ReLU())
        self.outputLayer = nn.Sequential(nn.Linear(1000,_out),nn.ReLU())
        self.reverseInputLayer = nn.Sequential(nn.Linear(1000,_in),nn.ReLU())
        self.reverseOutputLayer = nn.Sequential(nn.Linear(_out,1000),nn.ReLU())
        #self.s = nn.Parameter(torch.zeros(1))
        #self.si = nn.Parameter(torch.zeros(1))
        
    def split(self,x):
        x = x.view(-1)
        size = x.size()[0]//2
        x1 = x[:size]
        x2 = x[size:]
        return x1,x2
    
    def forward(self, x):
        x = self.inputLayer(x)
        for i in range(self.numLayers-2):
            x = self.hiddenLayer(x)
        x = self.outputLayer(x)
        return x
    
    def backward(self,x):
        x = self.reverseInputLayer(x)
        for i in range(self.numLayers-2):
            x = self.hiddenLayer(x)
        x = self.reverseOutputLayer(x)
        
    '''
    def forward(self,x):
        print(f"x size {x.size()}")
        x1,x2 = self.split(x)
        print(f"split into {x1.size()}, {x2.size()}")
        h1_1 = x1
        print(f"h1_1 size {h1_1.size()}")
        y = self.inputLayer(x1)
        print(f"y size {y.size()}")
        h1_2 = x2 + self.inputLayer(x1)
        print(f"h1_2 size {h1_2.size()}")
        #
        h2_1 = h1_2
        h2_2 = h1_1 + self.hiddenLayer(x2)
        #
        h3_1 = h2_1
        h3_2 = h2_2 + self.hiddenLayer(x1)
        #
        h4_1 = h3_2
        h4_2 = h3_1 + self.outputLayer(x2)
        print(f"h4_1 size {h4_1.size()}")
        print(f"h4_2 size {h4_2.size()}")
        #
        h4 = torch.cat((h4_1,h4_2))
        h = torch.exp(self.s)*h4
        return h
    '''
    '''
    def backward(self,y):
        y1,y2 = self.split(y)
        x1_1 = y1
        x1_2 = y2 - self.reverseInputLayer(y1)
        # 
        x2_1 = x1_2
        x2_2 = x1_1 - self.hiddenLayer(y2)
        #
        x3_1 = x2_1
        x3_2 = x2_2 - self.hiddenLayer(y1)
        #
        x4_1 = x3_2
        x4_2 = x3_1 - self.hiddenLayer(y2)
        #
        h4 = torch.cat((x4_1,x4_2))
        h = torch.exp(self.si)*h4
    '''

In [103]:
class NICE(nn.Module):
    def __init__(self,size, numLayers=5):
        super(NICE,self).__init__()
        self.numLayers = numLayers
        self.coupling = AdditiveCoupling(size//2,size//2)
        self.s = nn.Parameter(torch.zeros(1))
        #self.si = nn.Parameter(torch.zeros(1))
    
    def split(self,x):
        x = x.view(-1)
        size = x.size()[0]//2
        x1 = x[:size]
        x2 = x[size:]
        return x1,x2
        
    '''
    def forward(self,x):
        print(f"Nice input {x.size()}")
        x = x.view(-1)
        print(f"NICE view -1 {x.size()}")
        for i in range(self.numLayers):
            size = x.size()[0]
            #coup = AdditiveCoupling(size//2,siz)
            y = self.coupling(x)
            #y = coup(x)
            x = torch.cat((x,y))
        return x
    '''
    def forward(self,x):
        x1,x2 = self.split(x)
        h1_1 = x1
        h1_2 = x2 + self.coupling(x1)
        #
        h2_1 = h1_2
        h2_2 = h1_1 + self.coupling(x2)
        #
        h3_1 = h2_1
        h3_2 = h2_2 + self.coupling(x1)
        #
        h4_1 = h3_2
        h4_2 = h3_1 + self.coupling(x2)
        #
        h4 = torch.cat((h4_1,h4_2))
        h = torch.exp(self.s)*h4
        return h
    
        

In [104]:
from torch.distributions import Uniform, TransformedDistribution, MultivariateNormal
from torch.distributions.transforms import SigmoidTransform, AffineTransform
#Creating a logistic distribution
dimensions = 28**2
base_distribution = Uniform(torch.zeros(dimensions).to(device), torch.ones(dimensions).to(device))
transforms = [SigmoidTransform().inv, AffineTransform(loc=0, scale=1)]
logistic = TransformedDistribution(base_distribution, transforms)

#Creating a gaussian distribution
gaussian = MultivariateNormal(torch.zeros(dimensions).to(device), torch.eye(dimensions).to(device))
samples = logistic.sample([1000])
gauss = gaussian.sample([1000])

In [ ]:
epochs = 1
#net = AdditiveCoupling(28**2,28**2)
net = NICE(28**2)
lr = 10**(-3)
optimizer = optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-4)
prior = gaussian
for epoch in range(epochs):
    print(f"Epoch: {epoch}")
    for i,(img,label) in enumerate(mnist_data):
        print(f"Batch: {i}")
        optimizer.zero_grad()
        pred = net(img)
        #h = pred.sample()
        logprob = prior.log_prob(pred)
        loss = -torch.sum(logprob)
        loss.backward()
        optimizer.step()


Epoch: 0
Batch: 0
Batch: 1
Batch: 2
Batch: 3
Batch: 4
Batch: 5
Batch: 6
Batch: 7
Batch: 8
Batch: 9
Batch: 10
Batch: 11
Batch: 12
Batch: 13
Batch: 14
Batch: 15
Batch: 16
Batch: 17
Batch: 18
Batch: 19
Batch: 20
Batch: 21
Batch: 22
Batch: 23
Batch: 24
Batch: 25
Batch: 26
Batch: 27
Batch: 28
Batch: 29
Batch: 30
Batch: 31
Batch: 32
Batch: 33
Batch: 34
Batch: 35
Batch: 36
Batch: 37
Batch: 38
Batch: 39
Batch: 40
Batch: 41
Batch: 42
Batch: 43
Batch: 44
Batch: 45
Batch: 46
Batch: 47
Batch: 48
Batch: 49
Batch: 50
Batch: 51
Batch: 52
Batch: 53
Batch: 54
Batch: 55
Batch: 56
Batch: 57
Batch: 58
Batch: 59
Batch: 60
Batch: 61
Batch: 62
Batch: 63
Batch: 64
Batch: 65
Batch: 66
Batch: 67
Batch: 68
Batch: 69
Batch: 70
Batch: 71
Batch: 72
Batch: 73
Batch: 74
Batch: 75
Batch: 76
Batch: 77
Batch: 78
Batch: 79
Batch: 80
Batch: 81
Batch: 82
Batch: 83
Batch: 84
Batch: 85
Batch: 86
Batch: 87
Batch: 88
Batch: 89
Batch: 90
Batch: 91
Batch: 92
Batch: 93
Batch: 94
Batch: 95
Batch: 96
Batch: 97
Batch: 98
Batch: 99
B

Batch: 754
Batch: 755
Batch: 756
Batch: 757
Batch: 758
Batch: 759
Batch: 760
Batch: 761
Batch: 762
Batch: 763
Batch: 764
Batch: 765
Batch: 766
Batch: 767
Batch: 768
Batch: 769
Batch: 770
Batch: 771
Batch: 772
Batch: 773
Batch: 774
Batch: 775
Batch: 776
Batch: 777
Batch: 778
Batch: 779
Batch: 780
Batch: 781
Batch: 782
Batch: 783
Batch: 784
Batch: 785
Batch: 786
Batch: 787
Batch: 788
Batch: 789
Batch: 790
Batch: 791
Batch: 792
Batch: 793
Batch: 794
Batch: 795
Batch: 796
Batch: 797
Batch: 798
Batch: 799
Batch: 800
Batch: 801
Batch: 802
Batch: 803
Batch: 804
Batch: 805
Batch: 806
Batch: 807
Batch: 808
Batch: 809
Batch: 810
Batch: 811
Batch: 812
Batch: 813
Batch: 814
Batch: 815
Batch: 816
Batch: 817
Batch: 818
Batch: 819
Batch: 820
Batch: 821
Batch: 822
Batch: 823
Batch: 824
Batch: 825
Batch: 826
Batch: 827
Batch: 828
Batch: 829
Batch: 830
Batch: 831
Batch: 832
Batch: 833
Batch: 834
Batch: 835
Batch: 836
Batch: 837
Batch: 838
Batch: 839
Batch: 840
Batch: 841
Batch: 842
Batch: 843
Batch: 844

Batch: 1460
Batch: 1461
Batch: 1462
Batch: 1463
Batch: 1464
Batch: 1465
Batch: 1466
Batch: 1467
Batch: 1468
Batch: 1469
Batch: 1470
Batch: 1471
Batch: 1472
Batch: 1473
Batch: 1474
Batch: 1475
Batch: 1476
Batch: 1477
Batch: 1478
Batch: 1479
Batch: 1480
Batch: 1481
Batch: 1482
Batch: 1483
Batch: 1484
Batch: 1485
Batch: 1486
Batch: 1487
Batch: 1488
Batch: 1489
Batch: 1490
Batch: 1491
Batch: 1492
Batch: 1493
Batch: 1494
Batch: 1495
Batch: 1496
Batch: 1497
Batch: 1498
Batch: 1499
Batch: 1500
Batch: 1501
Batch: 1502
Batch: 1503
Batch: 1504
Batch: 1505
Batch: 1506
Batch: 1507
Batch: 1508
Batch: 1509
Batch: 1510
Batch: 1511
Batch: 1512
Batch: 1513
Batch: 1514
Batch: 1515
Batch: 1516
Batch: 1517
Batch: 1518
Batch: 1519
Batch: 1520
Batch: 1521
Batch: 1522
Batch: 1523
Batch: 1524
Batch: 1525
Batch: 1526
Batch: 1527
Batch: 1528
Batch: 1529
Batch: 1530
Batch: 1531
Batch: 1532
Batch: 1533
Batch: 1534
Batch: 1535
Batch: 1536
Batch: 1537
Batch: 1538
Batch: 1539
Batch: 1540
Batch: 1541
Batch: 1542
Batc

Batch: 2147
Batch: 2148
Batch: 2149
Batch: 2150
Batch: 2151
Batch: 2152
Batch: 2153
Batch: 2154
Batch: 2155
Batch: 2156
Batch: 2157
Batch: 2158
Batch: 2159
Batch: 2160
Batch: 2161
Batch: 2162
Batch: 2163
Batch: 2164
Batch: 2165
Batch: 2166
Batch: 2167
Batch: 2168
Batch: 2169
Batch: 2170
Batch: 2171
Batch: 2172
Batch: 2173
Batch: 2174
Batch: 2175
Batch: 2176
Batch: 2177
Batch: 2178
Batch: 2179
Batch: 2180
Batch: 2181
Batch: 2182
Batch: 2183
Batch: 2184
Batch: 2185
Batch: 2186
Batch: 2187
Batch: 2188
Batch: 2189
Batch: 2190
Batch: 2191
Batch: 2192
Batch: 2193
Batch: 2194
Batch: 2195
Batch: 2196
Batch: 2197
Batch: 2198
Batch: 2199
Batch: 2200
Batch: 2201
Batch: 2202
Batch: 2203
Batch: 2204
Batch: 2205
Batch: 2206
Batch: 2207
Batch: 2208
Batch: 2209
Batch: 2210
Batch: 2211
Batch: 2212
Batch: 2213
Batch: 2214
Batch: 2215
Batch: 2216
Batch: 2217
Batch: 2218
Batch: 2219
Batch: 2220
Batch: 2221
Batch: 2222
Batch: 2223
Batch: 2224
Batch: 2225
Batch: 2226
Batch: 2227
Batch: 2228
Batch: 2229
Batc

Batch: 2833
Batch: 2834
Batch: 2835
Batch: 2836
Batch: 2837
Batch: 2838
Batch: 2839
Batch: 2840
Batch: 2841
Batch: 2842
Batch: 2843
Batch: 2844
Batch: 2845
Batch: 2846
Batch: 2847
Batch: 2848
Batch: 2849
Batch: 2850
Batch: 2851
Batch: 2852
Batch: 2853
Batch: 2854
Batch: 2855
Batch: 2856
Batch: 2857
Batch: 2858
Batch: 2859
Batch: 2860
Batch: 2861
Batch: 2862
Batch: 2863
Batch: 2864
Batch: 2865
Batch: 2866
Batch: 2867
Batch: 2868
Batch: 2869
Batch: 2870
Batch: 2871
Batch: 2872
Batch: 2873
Batch: 2874
Batch: 2875
Batch: 2876
Batch: 2877
Batch: 2878
Batch: 2879
Batch: 2880
Batch: 2881
Batch: 2882
Batch: 2883
Batch: 2884
Batch: 2885
Batch: 2886
Batch: 2887
Batch: 2888
Batch: 2889
Batch: 2890
Batch: 2891
Batch: 2892
Batch: 2893
Batch: 2894
Batch: 2895
Batch: 2896
Batch: 2897
Batch: 2898
Batch: 2899
Batch: 2900
Batch: 2901
Batch: 2902
Batch: 2903
Batch: 2904
Batch: 2905
Batch: 2906
Batch: 2907
Batch: 2908
Batch: 2909
Batch: 2910
Batch: 2911
Batch: 2912
Batch: 2913
Batch: 2914
Batch: 2915
Batc

...

Batch: 53415
Batch: 53416
Batch: 53417
Batch: 53418
Batch: 53419
Batch: 53420
Batch: 53421
Batch: 53422
Batch: 53423
Batch: 53424
Batch: 53425
Batch: 53426
Batch: 53427
Batch: 53428
Batch: 53429
Batch: 53430
Batch: 53431
Batch: 53432
Batch: 53433
Batch: 53434
Batch: 53435
Batch: 53436
Batch: 53437
Batch: 53438
Batch: 53439
Batch: 53440
Batch: 53441
Batch: 53442
Batch: 53443
Batch: 53444
Batch: 53445
Batch: 53446
Batch: 53447
Batch: 53448
Batch: 53449
Batch: 53450
Batch: 53451
Batch: 53452
Batch: 53453
Batch: 53454
Batch: 53455
Batch: 53456
Batch: 53457
Batch: 53458
Batch: 53459
Batch: 53460
Batch: 53461
Batch: 53462
Batch: 53463
Batch: 53464
Batch: 53465
Batch: 53466
Batch: 53467
Batch: 53468
Batch: 53469
Batch: 53470
Batch: 53471
Batch: 53472
Batch: 53473
Batch: 53474
Batch: 53475
Batch: 53476
Batch: 53477
Batch: 53478
Batch: 53479
Batch: 53480
Batch: 53481
Batch: 53482
Batch: 53483
Batch: 53484
Batch: 53485
Batch: 53486
Batch: 53487
Batch: 53488
Batch: 53489
Batch: 53490
Batch: 53491

Batch: 54048
Batch: 54049
Batch: 54050
Batch: 54051
Batch: 54052
Batch: 54053
Batch: 54054
Batch: 54055
Batch: 54056
Batch: 54057
Batch: 54058
Batch: 54059
Batch: 54060
Batch: 54061
Batch: 54062
Batch: 54063
Batch: 54064
Batch: 54065
Batch: 54066
Batch: 54067
Batch: 54068
Batch: 54069
Batch: 54070
Batch: 54071
Batch: 54072
Batch: 54073
Batch: 54074
Batch: 54075
Batch: 54076
Batch: 54077
Batch: 54078
Batch: 54079
Batch: 54080
Batch: 54081
Batch: 54082
Batch: 54083
Batch: 54084
Batch: 54085
Batch: 54086
Batch: 54087
Batch: 54088
Batch: 54089
Batch: 54090
Batch: 54091
Batch: 54092
Batch: 54093
Batch: 54094
Batch: 54095
Batch: 54096
Batch: 54097
Batch: 54098
Batch: 54099
Batch: 54100
Batch: 54101
Batch: 54102
Batch: 54103
Batch: 54104
Batch: 54105
Batch: 54106
Batch: 54107
Batch: 54108
Batch: 54109
Batch: 54110
Batch: 54111
Batch: 54112
Batch: 54113
Batch: 54114
Batch: 54115
Batch: 54116
Batch: 54117
Batch: 54118
Batch: 54119
Batch: 54120
Batch: 54121
Batch: 54122
Batch: 54123
Batch: 54124

Batch: 54682
Batch: 54683
Batch: 54684
Batch: 54685
Batch: 54686
Batch: 54687
Batch: 54688
Batch: 54689
Batch: 54690
Batch: 54691
Batch: 54692
Batch: 54693
Batch: 54694
Batch: 54695
Batch: 54696
Batch: 54697
Batch: 54698
Batch: 54699
Batch: 54700
Batch: 54701
Batch: 54702
Batch: 54703
Batch: 54704
Batch: 54705
Batch: 54706
Batch: 54707
Batch: 54708
Batch: 54709
Batch: 54710
Batch: 54711
Batch: 54712
Batch: 54713
Batch: 54714
Batch: 54715
Batch: 54716
Batch: 54717
Batch: 54718
Batch: 54719
Batch: 54720
Batch: 54721
Batch: 54722
Batch: 54723
Batch: 54724
Batch: 54725
Batch: 54726
Batch: 54727
Batch: 54728
Batch: 54729
Batch: 54730
Batch: 54731
Batch: 54732
Batch: 54733
Batch: 54734
Batch: 54735
Batch: 54736
Batch: 54737
Batch: 54738
Batch: 54739
Batch: 54740
Batch: 54741
Batch: 54742
Batch: 54743
Batch: 54744
Batch: 54745
Batch: 54746
Batch: 54747
Batch: 54748
Batch: 54749
Batch: 54750
Batch: 54751
Batch: 54752
Batch: 54753
Batch: 54754
Batch: 54755
Batch: 54756
Batch: 54757
Batch: 54758

Batch: 55313
Batch: 55314
Batch: 55315
Batch: 55316
Batch: 55317
Batch: 55318
Batch: 55319
Batch: 55320
Batch: 55321
Batch: 55322
Batch: 55323
Batch: 55324
Batch: 55325
Batch: 55326
Batch: 55327
Batch: 55328
Batch: 55329
Batch: 55330
Batch: 55331
Batch: 55332
Batch: 55333
Batch: 55334
Batch: 55335
Batch: 55336
Batch: 55337
Batch: 55338
Batch: 55339
Batch: 55340
Batch: 55341
Batch: 55342
Batch: 55343
Batch: 55344
Batch: 55345
Batch: 55346
Batch: 55347
Batch: 55348
Batch: 55349
Batch: 55350
Batch: 55351
Batch: 55352
Batch: 55353
Batch: 55354
Batch: 55355
Batch: 55356
Batch: 55357
Batch: 55358
Batch: 55359
Batch: 55360
Batch: 55361
Batch: 55362
Batch: 55363
Batch: 55364
Batch: 55365
Batch: 55366
Batch: 55367
Batch: 55368
Batch: 55369
Batch: 55370
Batch: 55371
Batch: 55372
Batch: 55373
Batch: 55374
Batch: 55375
Batch: 55376
Batch: 55377
Batch: 55378
Batch: 55379
Batch: 55380
Batch: 55381
Batch: 55382
Batch: 55383
Batch: 55384
Batch: 55385
Batch: 55386
Batch: 55387
Batch: 55388
Batch: 55389

Batch: 55944
Batch: 55945
Batch: 55946
Batch: 55947
Batch: 55948
Batch: 55949
Batch: 55950
Batch: 55951
Batch: 55952
Batch: 55953
Batch: 55954
Batch: 55955
Batch: 55956
Batch: 55957
Batch: 55958
Batch: 55959
Batch: 55960
Batch: 55961
Batch: 55962
Batch: 55963
Batch: 55964
Batch: 55965
Batch: 55966
Batch: 55967
Batch: 55968
Batch: 55969
Batch: 55970
Batch: 55971
Batch: 55972
Batch: 55973
Batch: 55974
Batch: 55975
Batch: 55976
Batch: 55977
Batch: 55978
Batch: 55979
Batch: 55980
Batch: 55981
Batch: 55982
Batch: 55983
Batch: 55984
Batch: 55985
Batch: 55986
Batch: 55987
Batch: 55988
Batch: 55989
Batch: 55990
Batch: 55991
Batch: 55992
Batch: 55993
Batch: 55994
Batch: 55995
Batch: 55996
Batch: 55997
Batch: 55998
Batch: 55999
Batch: 56000
Batch: 56001
Batch: 56002
Batch: 56003
Batch: 56004
Batch: 56005
Batch: 56006
Batch: 56007
Batch: 56008
Batch: 56009
Batch: 56010
Batch: 56011
Batch: 56012
Batch: 56013
Batch: 56014
Batch: 56015
Batch: 56016
Batch: 56017
Batch: 56018
Batch: 56019
Batch: 56020